In [1]:
from manim import *
from numpy import linalg as npl
import math
import jupyter_capture_output

int_video_scene = " -v WARNING --progress_bar None --disable_caching integration_Scene"
int_image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s integration_Scene"

int_video_index_scene = " -v WARNING --progress_bar None --disable_caching integration_index_Scene"
int_image_index_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s integration_index_Scene"

Jupyter Capture Output v0.0.11


In [14]:
%%manim -qh --fps 60 $int_video_index_scene

class integration_index_Scene(Scene):
    def construct(self):
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        self.add(CVC)

        text_riemann = Tex(r"Riemann Sum: $\int_a^bf(x)dx\approx\sum_{i=1}^nf(x_i)\Delta x$", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
        text_riemann[0][21:33].set_color(BLUE)
        # text_riemann[0][34:46].set_color(BLUE)

        fxi_box = SurroundingRectangle(text_riemann[0][26:31], buff = .1, color = BLUE)
        dx_box = SurroundingRectangle(text_riemann[0][31:33], buff = .1, color = BLUE)


        # Integrierbare Funktion (abhängig von x)
        func_range = [2, 11]
        def integrable_function(x):
            return -1 + 3*x - 14*(x/4)**2 + 3.8*(x/4)**3

        
        # Riemann-Summe (abhängig von n)
        def riemann_squares(n):
            a = func_range[0]
            b = func_range[1]
            delta_x = (b - a) / n
            riemann_group = VGroup()
            for i in range(n):
                left = a + i * delta_x
                right = a + (i+1) * delta_x
                upper = integrable_function(right)
                polygon = Polygon(ax.c2p(left, 0), ax.c2p(left, upper), ax.c2p(right, upper), ax.c2p(right, 0), stroke_width = 1.5, color = BLUE, fill_color = BLUE, fill_opacity = 0.25, stroke_opacity = 1)
                riemann_group.add(polygon)
                #self.add(polygon)
            return riemann_group


        # Main: Coordinate System
        origin = [0, -0.5, 0]

        x_range = [-0.5, 3.75*PI, 1]
        y_range = [-4, 6, 1]

        x_length = 11
        y_length = 5.5

        x_dict = dict(zip([func_range[0], func_range[1]], [r"a", r"b"]))

        npla = NumberPlane(
            x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, 
            axis_config = {"include_numbers": False, "stroke_opacity": 0.25}, background_line_style = {"stroke_opacity": 0.25}, ).move_to(origin)
        ax = Axes(x_range = x_range, y_range = y_range, x_length = x_length, y_length = y_length, axis_config = {"stroke_width": 1, "stroke_opacity": 0.75}).move_to(origin).add_coordinates(x_dict)

        ax_xlabel = ax.get_x_axis_label(Tex(r"x", font_size = 36)).shift(0.25*DOWN + 0.125*RIGHT)
        ax_ylabel = ax.get_y_axis_label(Tex(r"y", font_size = 36)).shift(0.8125*LEFT + 0.5*DOWN)

        coordinate_system = VGroup(npla, ax, ax_xlabel, ax_ylabel)
        func = ax.plot(integrable_function, color = WHITE, stroke_width = 1.5, stroke_opacity = 1, x_range = [0, func_range[1]])

        fa_connect = DashedLine(ax.c2p(func_range[0], 0), ax.c2p(func_range[0], integrable_function(func_range[0])), stroke_color = WHITE, stroke_width = 1.5, stroke_opacity = 1)
        fb_connect = DashedLine(ax.c2p(func_range[1], 0), ax.c2p(func_range[1], integrable_function(func_range[1])), stroke_color = WHITE, stroke_width = 1.5, stroke_opacity = 1)

        # braces
        brace_rect = Polygon(
            ax.c2p(func_range[0], 0), 
            ax.c2p(func_range[0], integrable_function(func_range[0] + (func_range[1] - func_range[0]) / 10)), 
            ax.c2p(func_range[0] + (func_range[1] - func_range[0]) / 10, integrable_function(func_range[0] + (func_range[1] - func_range[0]) / 10)), 
            ax.c2p(func_range[0] + (func_range[1] - func_range[0]) / 10, 0))

        fxi_brace = Brace(brace_rect, color = BLUE, direction = np.array([-1, 0, 0]), buff = 0.2, sharpness = 2)
        dx_brace = Brace(brace_rect, color = BLUE, direction = np.array([0, -1, 0]), buff = 0.2, sharpness = 2)
        fxi_text = Tex(r"$f(x_1)$", font_size = 36, color = BLUE).next_to(fxi_brace, LEFT, buff = 0.2)
        dx_text = Tex(r"$\Delta x$", font_size = 36, color = BLUE).next_to(dx_brace, DOWN, buff = 0.1)
        fxi_brace_group = VGroup(fxi_brace, fxi_text)
        dx_brace_group = VGroup(dx_brace, dx_text)


        self.add(text_riemann, coordinate_system, func, fa_connect, fb_connect)
        self.wait(0.5)
        riemann_squares_pre = riemann_squares(10)
        self.play(Create(riemann_squares_pre), run_time = 1.5)
        self.wait(0.5)
        self.play(Create(fxi_box), Create(dx_box), run_time = 1.5)
        self.play(TransformFromCopy(fxi_box, fxi_brace_group), TransformFromCopy(dx_box, dx_brace_group), run_time = 1.5)
        self.wait(1.5)
        self.play(Uncreate(fxi_box), Uncreate(dx_box), FadeOut(fxi_brace_group), FadeOut(dx_brace_group), run_time = 1.5)
        self.wait(0.5)

        for i in range(11, 200, 1):
            riemann_squares_post = riemann_squares(i)
            self.play(FadeTransform(riemann_squares_pre, riemann_squares_post), run_time = 2 / i)
            riemann_squares_pre = riemann_squares_post

        self.wait(3)

Manim Community v0.19.0

[01/23/25 11:27:03] WARNING  The original total run_time of integration_index_Scene.play(), 0.0165289 ]8;id=439683;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=295452;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.0165289 ]8;id=69741;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=779874;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.0163934 ]8;id=834794;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=866355;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.0163934 ]8;id=759722;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=191439;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

...

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.0101523 ]8;id=808599;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=556624;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.010101  ]8;id=480332;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=496527;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.010101  ]8;id=866632;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=278237;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

[01/23/25 11:27:17] WARNING  The original total run_time of integration_index_Scene.play(), 0.0100503 ]8;id=383836;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=593710;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               

                    WARNING  The original total run_time of integration_index_Scene.play(), 0.0100503 ]8;id=201088;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=274596;file:///home/carlo/Desktop/single-projects/manim-demopage/.venv_demo/lib/python3.12/site-packages/manim/scene/scene.py#1044\1044]8;;\
                             seconds, is too short for the current frame rate of 60 FPS. Rendering                 
                             with the shortest possible total run_time of 0.0166667 seconds instead.               